In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import os, sys
import glob
from math import *
from scipy.ndimage import gaussian_filter1d
import scipy
from datetime import date



sns.set_style("ticks")
today = date.today()

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['gcd', 'radians', 'gamma', 'trunc', 'copysign', 'remainder', 'isclose', 'sqrt', 'fmod', 'frexp', 'hypot', 'isinf', 'log1p', 'log', 'exp', 'e', 'floor', 'nan', 'prod', 'log10', 'tan', 'expm1', 'log2', 'inf', 'sinh', 'isnan', 'ldexp', 'tanh', 'isfinite', 'cos', 'modf', 'pi', 'sin', 'cosh', 'degrees', 'ceil', 'fabs']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [3]:
# This is just boilerplate code to make the figures look good
sizer = 25
def format_figure(ax):
    '''Standard format to apply to all plots'''
    
    title_size = 30#sizer * 1.5
    for item in (ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(sizer)
    for item in [ax.title, ax.xaxis.label, ax.yaxis.label]:
        item.set_fontsize(title_size)    
    
    #plt.setp(ax.spines.values(), linewidth=2)
    ax.xaxis.set_tick_params(width=2)
    ax.yaxis.set_tick_params(width=2)
    
    return ax

def format_legend(legend):
    for label in legend.get_texts():
        label.set_fontsize(sizer*0.8)

    for label in legend.get_lines():
        label.set_linewidth(10)  # the legend line width
        
    return legend
        

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def wide_to_long(d):
    if np.ndim(d)==3:
        l=np.shape(d)[1]
        split_data=np.concatenate([d[:,range(0,l,2),:],d[:,range(0,l,2),:]],0)
    elif np.ndim(d)==2:
        l=np.shape(d)[0]
        split_data=np.stack([d[range(0,l,2),:],d[range(0,l,2),:]],0)

    newshape_data=np.swapaxes(split_data,0,1)
    return newshape_data

sns.set_style("ticks")


# Set your default plot settings here.
what_for=0
if what_for==0: #for computer display
    leg_pos=0 #0 for upper left, 1 for upper right
    l=None #width of lines
    e=None #width of errorbars
    ec=None #color of errorbars
    y=1.6 #ylim
elif what_for==1: #for presentation
    leg_pos=1 #0 for upper left, 1 for upper right
    l=4 #width of lines
    e=1.5 #width of errorbars
    ec="black"
    y=1.5 #ylim

In [4]:
def make_longdata(data):
    for r in range(data.shape[0]):
        #if np.ndim(data.iloc[r]['data'])==2:
        #    data.iloc[r]['data']=data.iloc[r]['data'][:,np.newaxis,:]
        index=pd.MultiIndex.from_product([concs,
                                          range(np.shape(data.iloc[r]['data'])[1]),
                                          range(10,(10*np.shape(data.iloc[r]['data'])[2]+1),10),
                                          [data.iloc[r]['phenotype']],
                                          [data.iloc[r]['drug']],
                                          [data.iloc[r]['bio_rep']]
                                         ],
                        names=['concentration','tech_rep', 'timepoint','phenotype','drug','bio_rep'])
        t=(pd.DataFrame({'mobility_score':data.iloc[r]['data']
                         .flatten()},index=index))
        if r==0:
            final=t
        else:
            final=pd.concat([final,t])
        final=(final.mask(final.mobility_score==0,0.00001))
    return(final)

def N2_norm(data):
    norm=(data
          .query('phenotype=="WT"|phenotype=="GFP"|phenotype=="0.8% DMSO"')
          .query('concentration==0')
          .groupby(['bio_rep','timepoint','drug'])
          .mean()
          .reset_index()
         .assign(norm_method='N2_norm'))
    r=(data
       .reset_index()
       .merge(norm,on=["bio_rep","timepoint",'drug']))
    return(r)

def con_norm(data):
    norm=(data
          .query('concentration==0')
          .groupby(['bio_rep','timepoint','phenotype','drug'])
          .mean()
          .reset_index()
         .assign(norm_method='con_norm'))
    r=(data
       .reset_index()
       .merge(norm,on=["bio_rep","timepoint","phenotype",'drug']))
    return(r)

def dose_norm(data):
    norm=(data
          .query('phenotype=="WT"|phenotype=="GFP"|phenotype=="0.8% DMSO"')
          .groupby(['bio_rep','timepoint','concentration','drug'])
          .mean()
          .reset_index()
         .assign(norm_method='dose_norm'))
    r=(data
       .reset_index()
       .merge(norm,on=["bio_rep","timepoint","concentration",'drug']))
    return(r)

def T0_norm(data):
    norm=(data
          .query('phenotype=="WT"|phenotype=="GFP"|phenotype=="0.8% DMSO"')
          .query('timepoint==10')
          .query('concentration==0')
          .groupby(['bio_rep','drug'])
          .mean()
          .reset_index()
         .assign(norm_method='T0_norm'))
    r=(data
       .reset_index()
       .merge(norm,on=["bio_rep",'drug']))
    return(r)


def all_norm(data):
    norm=[]
    for n in [N2_norm,con_norm,dose_norm,T0_norm]:
        norm.append(n(data))
    return(pd.concat(norm))

In [5]:
in_dir='/Users/margotlautenswork/Documents/Data/'

data1 = np.load(in_dir+'2018/08Aug18/09aug01/N2_kynu-1_cysl_09aug.npy')
data2 = np.load(in_dir+'2018/08Aug18/15aug02/N2_hif1_coq2x2_coq2x3_KCN_15aug001.npy')
data3 = np.load(in_dir+'2018/08Aug18/17aug01/N2_egl-9_mev-1_nhr-57_17aug.npy')
data4 = np.load(in_dir+'2018/08Aug18/18aug01/N2_kynu1_kynu-1egl9_nhr57egl9_18aug001.npy')
data5 = np.load(in_dir+'2018/08Aug18/23aug01/N2_coq2x2_coq2x3_L34.npy')
data6 = np.load(in_dir+'2018/08Aug18/24aug01/KCN_aug24.npy')
data7 = np.load(in_dir+'2018/08Aug18/28aug01/N2_coqx2_coq2x3_hif1_nhr57_kynu1_28aug.npy')
data8 = np.load(in_dir+'2018/08Aug18/30aug01/KCN_expt_30aug.npy')
data9 = np.load(in_dir+'2018/08Aug18/31aug01/KCN_expt_31aug.npy')
data10 = np.load(in_dir+'2018/09Sep18/12sep01/sep12_KCN.npy')
data11 = np.load(in_dir+'2018/09Sep18/19sep01/sep19_N2_egl9_hif1_nhr57_coq2x3_icl1_kynu1_mev1.npy')
data12 = np.load(in_dir+'2018/10Oct18/03oct01/N2_kynu-1_KCN_FCCP_NAC_Rot_03oct18001.npy')
data13 = np.load(in_dir+'2018/10Oct18/20oct01/N2_idhb1_idh2_kynu1_pre33_PHX245_RB1297_RB1484_RB2485_20oct.npy')
data14 = np.load(in_dir+'2019/06Jun19/01jun01/OP50_kynu1_484_idh1_mmcm1_KCN_01jun001.npy')
data15 = np.load(in_dir+'2019/06Jun19/04jun01/June4_RNAi_15h_KCN.npy')
data16 = np.load(in_dir+'2019/06Jun19/05jun01/2019June5.npy')
data17 = np.load(in_dir+'2019/06Jun19/08jun01/June8_MR507_RB574_clk1_egl9hif1_unc53.npy')
data18 = np.load(in_dir+'2019/06Jun19/11jun01/June11_GFP_drp1_F27_let721_pept1.npy')
data19 = np.load(in_dir+'2019/06Jun19/13jun01/June13_RNAi_KCN_15h.npy')
data20 = np.load(in_dir+"2019/07Jul19/23jul01/Jul23_N2_PHX243_CE541_GFP_sbp1_gpd3.npy")
data21 = np.load(in_dir+"2019/07Jul19/27jul01/Jul27_N2_MR507_RB574_PHX243_diet_16.5h.npy")
data22 = np.load(in_dir+"2019/07Jul19/30jul01/Jul30_GFP_EV_pept1_pept1d2_kynu1GFP_kynupept1.npy")
data23 = np.load(in_dir+"2019/07Jul19/30jul02/Jul30_N2_MR507_RB574_PHX243_N2_MR507_RB574_16.5h.npy")
data24 = np.load(in_dir+"2019/08Aug19/02aug01/Aug2_N2_MR507_RB574_N2mixed_RB574mixed.npy")
data25 = np.load(in_dir+"2019/08Aug19/13aug01/Aug13_GFP_drp1_RNAi_PZP_kynu1PD4470.npy")
data26 = np.load(in_dir+'2019/09Sep19/18sep01/Sep18_KCN.npy')
data27 = np.load(in_dir+'2019/10Oct19/03oct01/N2old_N2June_idh-2_idhg-2_KCN_03oct19_01.npy')
data28 = np.load(in_dir+'2019/10Oct19/23oct01/RNAi_23oct19_001.npy')
data29 = np.load(in_dir+'2019/10Oct19/23oct02/RNAi_N2_23oct19_002.npy')
data30 = np.load(in_dir+'2019/11Nov19/09nov01/N2_KCN_18hr_09nov19_001.npy')
data31 = np.load(in_dir+'2019/11Nov19/09nov02/RNAi_KCN_18hr_09nov19_002.npy')
data32 = np.load(in_dir+'Several/wact-11/N2_wact11_KCN_18hr_Nov1318.npy')
data33 = np.load(in_dir+'Several/wact-11/KCN_wact11_DoubleDoseResponse_18hr_Jan819.npy')
data34 = np.load(in_dir+'Several/wact-11/KCN_wact11_DoubleDoseResponse_18hr_Jan1119.npy')
data35 = np.load(in_dir+'2019/11Nov19/10nov01/RNAi_KCN_18hr_10nov19_001.npy')
data36 = np.load(in_dir+'2019/June/Apr1_N2_daf2_rhy1_salt.npy')
data37 = np.load(in_dir+'2019/June/Apr3_N2x2_daf2_rhy1.npy')
data38 = np.load(in_dir+'2019/June/Mar12_AF25_N2_kynu1_rhy1_kynu1rhy1_16h.npy')
data39 = np.load(in_dir+'2019/June/Oct12_N2_AF25_VC345_RB1036_GR1307_RB1297.npy')
data40 = np.load(in_dir+'2019/June/dec1_N2_ZG31_PHX242_kynu1.npy')
data41 = np.load(in_dir+'2019/June/Nov13_N2_GUC01_GUC02_AF25_KQ6_KQ1366_16h.npy')
data42 = np.load(in_dir+'2019/June/Dec12_N2_BR2742_JT528_KQ6_KQ1366_RB925.npy')
data43 = np.load(in_dir+'2019/June/Jan10_N2_KQ6_KQ1366_SJ17_SJ30_N2_16.5h.npy')
data44 = np.load(in_dir+'2019/June/Oct10_15h_KCN_N2_GUC01_GUC02.npy')
data45 = np.load(in_dir+'2019/June/Oct16_N2_TM1420_VC393_SJ30_15h.npy')
data46 = np.load(in_dir+'2019/June/Mar2_N2_CU6372_SJ17_SJ30_4xRQS_CB1003_16h.npy')
data47 = np.load(in_dir+'2019/June/Oct24_N2_TM1420_VC393_AF25_BR2742_SJ17.npy')
data48 = np.load(in_dir+'2019/June/Dec20_VC345_JT529_JT528_JT513_DA631_BR2742_N2_16.5h_KCN.npy')
data49 = np.load(in_dir+'2019/June/Oct19_N2_AF25_VC345_BR2742.npy')
data50 = np.load(in_dir+'2019/June/Jul16_RNAi_16h.npy')
data51 = np.load(in_dir+'2019/June/Aug116_EV_GFP_dhod1_kynu1_pept1.npy')
data52 = np.load(in_dir+'2019/June/Aug26_GFP_daf15_dhod1_F36_prdh1_rheb1_15h.npy')
data53 = np.load(in_dir+'2019/June/2020/Aug8/Aug8_N2_ZG31_MT1216_RB766_mce1_N2.npy')
data54 = np.load(in_dir+'2019/June/2020/Aug23/Aug23_N2_ZG31_RB766_GFP_F36A23_mdh1.npy')
data55 = np.load(in_dir+'2019/June/2020/Aug26/Aug26_15h_KCN.npy')
data56 = np.load(in_dir+'2019/June/2020/Jun20_1/June20_16h_KCN_kynu1rhy1.npy')
data57 = np.load(in_dir+'2019/June/2020/Jun27_2/Jun17_mutants.npy')
data58 = np.load(in_dir+'2019/June/2020/Sep2/Sep2_N2_ZG31_OW715_OW717.npy')
data59 = np.load(in_dir+'2019/June/2020/new_scope/Sep23/Sep23_N2_OW715_OW716_OW717.npy')
data60 = np.load(in_dir+'2019/June/2020/Sep24/Sep24_N2_OW715_OW716_OW717.npy')
data61 = np.load(in_dir+'2019/June/2020/Jun20_2/June20_15h_KCN_RNAi.npy')
data62 = np.load(in_dir+'2019/June/2020/Jun27_1/Jun17_RNAi.npy')
data63 = np.load(in_dir+'2019/June/2020/Jun30/Jun30_RNAi001.npy')
data64 = np.load(in_dir+'2019/June/2020/Jul4/Jul4_RNAi002.npy')
data65 = np.load(in_dir+'2019/June/2020/Jul7/Jul7_RNAi.npy')
data66 = np.load(in_dir+'2019/June/2020/Jul11/Jul11_RNAi.npy')
data67 = np.load(in_dir+'2019/June/2020/Jul18_1/Jul18_RNAi_1.npy')
data68 = np.load(in_dir+'2019/June/2020/Jul18_2/Jul18_RNAi_2.npy')
data69 = np.load(in_dir+'2019/June/2020/Jul22/Jul22_RNAi.npy')
data70 = np.load(in_dir+'2019/June/2020/Jul25_1/Jul25_RNAi_1.npy')
data71 = np.load(in_dir+'2019/June/2020/Jul25_2/Jul25_RNAi_2.npy')
data72 = np.load(in_dir+'2019/June/2019/Oct2/Oct2_15h_KCN.npy')
data73 = np.load(in_dir+'2019/June/2019/Oct5/Oct5_15.5h_N2_GUC01_GUC02_N2_kynu1_kynu1rhy1002.npy')
data74 = np.load(in_dir+'2019/June/2020/Mar12/Mar12_AF25_N2_kynu1_rhy1_kynu1rhy1_16h.npy')
data75 = np.load(in_dir+'2019/June/Oct1_N2_OW715_OW716_OW717.npy')
data76 = np.load(in_dir+'2019/June/Aug1_RNAi.npy')




save_dir=in_dir+'Several/18hr Assay/'

titl='fullset'
date='aug2018_oct2020'

concs = [0,200,300]

In [6]:


rep1=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "cysl-1",
        "cysl-2"
    ]),
    'data':[
    data1[4:7,0:4,:], #N2
    data1[4:7,3:6,:], #kynu-1
    data1[4:7,6:9,:], #cysl-1
    data1[4:7,9:12,:], #cysl-2
    ],
    'drug':"M9",
    'bio_rep':1
})

rep2=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "hif-1",
        "coq-2x3"]),
    'data':[  
    data2[4:7,0:4,:], #N2
    data2[4:7,3:5,:], #hif1
    data2[4:7,9:12,:], #coq-2x3
    ],
    'drug':"M9",
    'bio_rep': 2,
              
             })

rep3=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "mev-1",
        "egl-9",
        "nhr-57"
    ]),
    'data':[
    data3[4:7,0:3:2,:], #N2
    data3[4:7,6:9,:], #mev1
    data3[4:7,3:6:2,:], #egl9
    data3[4:7,9:12,:], #nhr57
    ],
    'drug':"M9",
    'bio_rep':3
})

rep4=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "egl-9;kynu-1",
        "egl-9;nhr-57"
    ]),
    'data':[
    data4[4:7,0:4,:], #N2
    data4[4:7,3:6,:], #kynu-1
    data4[4:7,6:9,:], #kynu-1;egl-9
    data4[4:7,9:12,:], #nhr-57;egl-9
    ],
    'drug':"M9",
    'bio_rep':4
})

rep5=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "coq-2x3"
    ]),
    'data':[
    data5[4:7,0:4,:], #N2 DMSO
    data5[4:7,6:9,:], #coq-2x3 DMSO
    ],
    'drug':"0.8% DMSO",
    'bio_rep':5
})

rep6=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "egl-9",
        "nhr-57",
        "egl-9;kynu-1",
        "egl-9;nhr-57"
    ]),
    'data':[
    data6[:3,0:4,:], #N2 16
    data6[4:7,3:6,:], #egl-9
    data6[4:7,6:9,:], #nhr-57
    data6[:3,9:12,:], #kynu-1;egl-9 
    data6[4:7,9:12,:], #nhr-57;egl-9
    ],
    'drug':"M9",
    'bio_rep':6
})


rep7=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "hif-1",
        "nhr-57",
        "coq-2x3"
    ]),
    'data':[
    data7[4:7,0:2,:], #N2 0.8% DMSO
    data7[4:7,8:10,:], #kynu-1 0.8% DMSO
    data7[4:7,6:8,:], #hif-1 0.8% DMSO
    data7[4:7,10:12,:], #nhr-57 0.8% DMSO
    data7[4:7,4:6,:], #coq-2 triple 0.8% DMSO
    ],
    'drug':"0.8% DMSO",
    'bio_rep':7
})

rep8=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "hif-1",
        "egl-9",
        "nhr-57",
        "egl-9;kynu-1",
        "egl-9;nhr-57"
    ]),
    'data':[
    wide_to_long(data8[0,(0,1,6,7,10,11),:]), #N2
    wide_to_long(data8[3,(0,1,6,7,10,11),:]), #kynu-1
    wide_to_long(data8[2,(0,1,6,7,10,11),:]), #hif-1
    wide_to_long(data8[1,(0,1,6,7,10,11),:]), #egl-9
    wide_to_long(data8[4,(0,1,6,7,10,11),:]), #nhr-57
    wide_to_long(data8[5,(0,1,6,7,10,11),:]), #egl-9;kynu-1
    wide_to_long(data8[6,(0,1,6,7,10,11),:]), #egl-9;nhr-57
    ],
    'drug':"M9",
    'bio_rep':8
})

rep9=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "egl-9;kynu-1",
        "nhr-57",
        "egl-9;nhr-57",
        "alh-6(wr1)"
    ]),
    'data':[
    wide_to_long(data9[0,(0,1,6,7,10,11),:]), #N2
    wide_to_long(data9[2,(0,1,6,7,10,11),:]), #kynu-1
    wide_to_long(data9[3,(0,1,6,7,10,11),:]), #kynu-1;egl-9
    wide_to_long(data9[4,(0,1,6,7,10,11),:]), #nhr-57
    wide_to_long(data9[5,(0,1,6,7,10,11),:]), #nhr-57;egl-9
    wide_to_long(data9[6,(0,1,6,7,10,11),:]), #pre-1
    ],
    'drug':"M9",
    'bio_rep':9
})

rep10=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "mev-1",
        "hif-1",
        "egl-9",
        "nhr-57",
        "egl-9;mev-1"
    ]),
    'data':[
    data10[:3,0:4,:], #N2
    data10[4:7,6:9,:], #kynu-1
    data10[:3,6:9,:], #mev-1
    data10[4:7,3:6,:], #hif-1
    data10[:3,3:6,:], #egl-9
    data10[4:7,9:12,:], #nhr-57
    data10[:3,9:12,:], #egl-9;mev-1
    ],
    'drug':"M9",
    'bio_rep':10
})

rep11=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "mev-1",
        "hif-1",
        "egl-9",
        "nhr-57",
        "coq-2x3",
        "icl-1"
    ]),
    'data':[
    data11[:3,0:4,:], #N2
    data11[4:7,6:9,:], #kynu-1
    data11[4:7,9:12,:], #mev-1
    data11[:3,6:9,:], #hif-1
    data11[:3,3:6,:], #egl-9
    data11[:3,9:12,:], #nhr-57
    data11[4:7,0:4,:], #coq-2x3
    data11[4:7,3:6,:], #icl-1
    ],
    'drug':"M9",
    'bio_rep':11
})

rep12=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "WT",
        "kynu-1",
        "WT",
        "kynu-1",
        "WT",
        "kynu-1",
    ]),
    'data':[
    data12[:3,1:4,:],#N2
    data12[:3,3:6,:],#kynu-1
    data12[:3,6:9,:],#N2 w 1µM FCCP
    data12[:3,9:12,:],#kynu-1 w 1µM FCCP
    data12[4:7,0:4,:],#N2 w 50µM NAC
    data12[4:7,3:6,:],#kynu-1 w 50µM NAC
    data12[4:7,6:9,:],#N2 w 12.5µM Rotenone
    data12[4:7,9:12,:],#kynu-1 w 12.5µM Rotenone
    ],
    'drug':pd.Categorical([
        "0.8% DMSO",
        "0.8% DMSO",
        "1µM FCCP",
        "1µM FCCP",
        "50µM NAC",
        "50µM NAC",
        "12.5µM Rotenone",
        "12.5µM Rotenone"
    ]),
    'bio_rep':12
})

rep13=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "idhb-1",
        "idh-2",
        "dld-1",
        "acdh-7",
        "rhy-1",
        "mmcm-1",
        "sqrd-1"
    ]),
    'data':[
    wide_to_long(np.swapaxes(data13[(0,4,1,5,2,6),0:2,:19],0,1)),#N2
    wide_to_long(np.swapaxes(data13[(0,4,1,5,2,6),2:4,:19],0,1)),#idhb1
    wide_to_long(np.swapaxes(data13[(0,4,1,5,2,6),4:6,:19],0,1)),#idh2
    wide_to_long(data13[np.newaxis,(0,4,1,5,2,6),7,:19]),#dld-1(wr4)
    wide_to_long(data13[np.newaxis,(0,4,1,5,2,6),8,:19]),#acdh-7
    wide_to_long(data13[np.newaxis,(0,4,1,5,2,6),9,:19]),#rhy-1
    wide_to_long(data13[np.newaxis,(0,4,1,5,2,6),10,:19]),#mmcm-1
    wide_to_long(data13[np.newaxis,(0,4,1,5,2,6),11,:19]),#sqrd-1
    ],
    'drug':"M9",
    'bio_rep':13
})

rep14=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "idh-1",
        "mmcm-1"
    ]),
    'data':[
    data14[(6,4,2),0:4,:],#N2
    data14[(6,4,2),3:6,:],#kynu-1
    data14[(6,4,2),8:10,:],#idh-1
    data14[(6,4,2),11,np.newaxis,:],#mmcm-1
    ],
    'drug':"M9",
    'bio_rep':14
})

rep15=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "aak-1",
        "f27d4.1",
        "hyl-2",
        "let-721",
        "rict-1"
    ]),
    'data':[
    data15[(7,4,2),0:2,:],#GFP
    data15[(7,4,2),2:4,:],#aak-1
    data15[(7,4,2),4:6,:],#F27D4.1
    data15[(7,4,2),6:8,:],#hyl-2
    data15[(7,4,2),8:10,:],#let-721
    data15[(7,4,2),10:,:],#rict-1
    ],
    'drug':"M9",
    'bio_rep':15
})

rep16=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "mmcm-1",
        "acdh-7",
        "alh-6(wr1)",
        "kynu-1"
    ]),
    'data':[
    data16[(7,4,2),0:4,:],#N2
    data16[(7,4,2),3:6,:],#mmcm-1
    data16[(7,4,2),6:8,:],#acdh-7
    data16[(7,4,2),8:10,:],#pre-1
    data16[(7,4,2),10:,:],#kynu-1
    ],
    'drug':"M9",
    'bio_rep':16
})

rep17=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "aak-2(rr48)",
        "aak-2(ok524)",
        "clk-1",
        "egl-9;hif-1",
        "unc-53"
    ]),
    'data':[
    data17[(7,4,2),0:2,:],#WT
    data17[(7,4,2),2:4,:],#aak-2(rr48)
    data17[(7,4,2),4:6,:],#aak-2(ok524)
    data17[(7,4,2),6:8,:],#clk-1
    data17[(7,4,2),8:10,:],#egl-9;hif-1
    data17[(7,4,2),10:,:],#unc-53
    ],
    'drug':"M9",
    'bio_rep':17
})

rep18=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "drp-1",
        "f27d4.1",
        "let-721",
        "pept-1"
    ]),
    'data':[
    data18[[[7],[4],[2]],[0,1,10,11],:],#GFP
    data18[(7,4,2),2:4,:],#drp-1
    data18[(7,4,2),4:7,:],#F27D4.1
    data18[(7,4,2),7,np.newaxis,:],#let-721
    data18[(7,4,2),8:10,:],#pept-1
    ],
    'drug':"M9",
    'bio_rep':18
})

rep19=pd.DataFrame({
    'phenotype':pd.Categorical([
        "drp-1",
        "eat-3",
        "f27d4.1",
        "GFP",
        "hyl-2",
        "let-721"
    ]),
    'data':[
    data19[(7,4,2),0:2,:],#drp-1
    data19[(7,4,2),2:4,:],#eat-3
    data19[(7,4,2),4:6,:],#F27D4.1
    data19[[[7],[4],[2]],[6,7,11],:],#GFP
    data19[(7,4,2),8:10,:],#hyl-2
    data19[(7,4,2),10,np.newaxis,:],#let-721
    ],
    'drug':"M9",
    'bio_rep':19
})

rep20=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "vf13d12l.3.1",
        #"sbp-1"
    ]),
    'data':[
    data20[(7,4,2),0:2,:],#WT
    data20[(7,4,2),2:4,:],#aak-2(ok524)
    #data20[(7,4,2),4:6,:],#sbp-1
    ],
    'drug':"M9",
    'bio_rep':20
})

rep21=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "aak-2(rr48)",
        "aak-2(ok524)",
        "vf13d12l.3.1"
    ]),
    'data':[
    data21[(7,4,2),0:2,:],#WT
    data21[(7,4,2),2:4,:],#aak-2(rr48)
    data21[(7,4,2),4:6,:],#aak-2(ok524)
    data21[(7,4,2),6,np.newaxis,:],#VF13D12L.3.1
    ],
    'drug':"M9",
    'bio_rep':21
})

rep22=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "pept-1",
        "sbp-1",
        "kynu-1;GFP",
        "kynu-1;pept-1"
    ]),
    'data':[
    data22[(7,4,2),0:2,:],#GFP
    data22[(7,4,2),4:6,:],#pept-1
    data22[(7,4,2),8:10,:],#sbp-1
    data22[(7,4,2),10,np.newaxis,:],#kynu-1 GFP
    data22[(7,4,2),11,np.newaxis,:],#kynu-1 pept-1
    ],
    'drug':"M9",
    'bio_rep':22
})

rep23=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "aak-2(rr48)",
        "aak-2(ok524)",
        "vf13d12l.3.1"
    ]),
    'data':[
    data23[(7,4,2),0:2,:],#N2
    data23[(7,4,2),2:4,:],#aak-2(rr48)
    data23[(7,4,2),4:6,:],#aak-2(ok524)
    data23[(7,4,2),6:8,:],#VF13D12L.3.1
    ],
    'drug':"M9",
    'bio_rep':23
})

rep24=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "aak-2(rr48)",
        "aak-2(ok524)",
    ]),
    'data':[
    data24[(7,4,2),0:2,:],#N2
    data24[(7,4,2),2:4,:],#aak-2(rr48)
    data24[(7,4,2),4:6,:],#aak-2(ok524)
    ],
    'drug':"M9",
    'bio_rep':24
})

rep25=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "drp-1",
        "sbp-1",
        "GFP",
        "drp-1",
    ]),
    'data':[
    data25[(7,4,2),0:2,:],#GFP
    data25[(7,4,2),2:4,:],#drp-1
    data25[(7,4,2),4:6,:],#sbp-1
    data25[(7,4,2),8:10,:],#GFP PZP
    data25[(7,4,2),10:,:],#drp-1 PZP
    ],
    'drug':pd.Categorical([
        "0.8% DMSO",
        "0.8% DMSO",
        "0.8% DMSO",
        "PZP",
        "PZP"
    ]),
    'bio_rep':25
})


rep26=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "dld-1",
        "pck-1",
        "hpd-1",
        "idhb-1",
        "idh-2"
    ]),
    'data':[
    hstack((data26[(3,2,1),0:2,:],data26[(7,6,5),0:2,:])), #N2
    hstack((data26[(3,2,1),2:4,:],data26[(7,6,5),2:4,:])), #dld-1
    hstack((data26[(3,2,1),4:6,:],data26[(7,6,5),4:6,:])), #pck-1
    hstack((data26[(3,2,1),6:8,:],data26[(7,6,5),6:8,:])), #hpd-1
    hstack((data26[(3,2,1),8:10,:],data26[(7,6,5),8:10,:])), #idhb-1
    hstack((data26[(3,2,1),10:12,:],data26[(7,6,5),10:12,:])), #idh-2
    ],
    'drug':"M9",
    'bio_rep':26
})

rep27=pd.DataFrame({
    'phenotype':pd.Categorical({
        "WT",
        "idh-2",
        "idhg-2"
    }),
    'data':[
        data27[[[7],[4],[2]],[0,4,8,10],:], #WT old
        data27[(7,4,2),2:4,:], #idh-2
        data27[(7,4,2),6:8:], #idhg-2
    ],
    'drug':"M9",
    'bio_rep':27
})

rep28=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "fah-1",
        "gspd-1",
        "tkt-1",
        "y57G11C.3",
        "laat-1",
    ]),
    'data':[
        hstack((data28[[3,2,1],0:2,:],data28[[7,6,5],0:2,:])), #GFP
        hstack((data28[[3,2,1],2:4,:],data28[[7,6,5],2:4,:])), #fah-1
        data28[[3,2,1],4:6,:], #gspd-1
        hstack((data28[[3,2,1],6,np.newaxis,:],data28[[7,6,5],7,np.newaxis,:])),  #tkt-1
        hstack((data28[[3,2,1],9,np.newaxis,:],data28[[7,6,5],8,np.newaxis,:])),#Y57G11.3
        data28[[7,6,5],10,np.newaxis,:],  #laat-1
    ],
    'drug':"M9",
    'bio_rep':28
})

rep29=pd.DataFrame({
    'phenotype':pd.Categorical([
        "fah-1",
        "laat-1",
        #"tkt-1",
        "y57G11C.3",
        "WT",
        "aass-1",
        "pck-1"
    ]),
    'data':[
        data29[[3,2,1],3,np.newaxis,:],  
        data29[[3,2,1],4,np.newaxis,:],  
        #hstack((data29[[3,2,1],5,np.newaxis,:],data29[[7,6,5],5,np.newaxis,:])),  
        data29[[3,2,1],6,np.newaxis,:], 
        data29[[3,2,1],7:9,:],
        hstack((data29[[3,2,1],9,np.newaxis,:],data29[[7,6,5],9,np.newaxis,:])), 
        hstack((data29[[3,2,1],10:,:],data29[[7,6,5],10:,:]))
    ],
    'drug':"M9",
    'bio_rep':29
})

rep30=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "idh-1",
        "idhg-2",
        "aass-1",
        "pck-1",
        "hpd-1"
    ]),
    'data':[
        hstack((data30[[3,2,1],0:2,:],data30[[7,6,5],0:2,:])), #N2
        data30[[3,2,1],2:4,:], #idh-1
        hstack((data30[[3,2,1],4:6,:],data30[[7,6,5],4:6,:])), #idhg-2
        data30[[7,6,5],6:8,:], #aass-1
        hstack((data30[[3,2,1],8:10,:],data30[[7,6,5],8:10,:])), #pck-1
        hstack((data30[[3,2,1],11,np.newaxis,:],data30[[7,6,5],10:12,:])), #hpd-1
    ],
    'drug':"M9",
    'bio_rep':30
})


rep31=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "fah-1",
        "laat-1",
        "pmt-1",
        "tkt-1",
    ]),
    'data':[
        data31[[7,6,5],1,np.newaxis,:], #GFP
        hstack((data31[[3,2,1],3,np.newaxis,:],data31[[7,6,5],2:4,:])), #fah-1
        hstack((data31[[3,2,1],5,np.newaxis,:],data31[[7,6,5],4:6,:])), #laat-1
        hstack((data31[[3,2,1],6,np.newaxis,:],data31[[7,6,5],6:8,:])), #pmt-1
        hstack((data31[[3,2,1],8,np.newaxis,:],data31[[7,6,5],8:10,:])), #tkt-1
    ],
    'drug':"M9",
    'bio_rep':31
})

rep32=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "WT"
    ]),
    'data':[
        hstack((data32[7,[5,2,0],np.newaxis,:],data32[7,[11,8,6],np.newaxis,:])),
        data32[3,[5,2,0],np.newaxis,:],
    ],
    'drug':pd.Categorical([
        "0.8% DMSO",
        "10µM wact-11"
    ]),
    'bio_rep':32
})

rep33=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "WT"
    ]),
    'data':[
        hstack((data33[7,[5,2,0],np.newaxis,:],data33[7,[11,8,6],np.newaxis,:])),
        hstack((data33[3,[5,2,0],np.newaxis,:],data33[3,[11,8,6],np.newaxis,:])),
    ],
    'drug':pd.Categorical([
        "0.8% DMSO",
        "10µM wact-11"
    ]),
    'bio_rep':33
})

rep34=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "WT"
    ]),
    'data':[
        hstack((data34[7,[5,2,0],np.newaxis,:],data34[7,[11,8,6],np.newaxis,:])),
        hstack((data34[3,[5,2,0],np.newaxis,:],data34[3,[11,8,6],np.newaxis,:])),
    ],
    'drug':pd.Categorical([
        "0.8% DMSO",
        "10µM wact-11"
    ]),
    'bio_rep':34
})

rep35=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "gspd-1",
        "fah-1",
        "pmt-1",
        "tkt-1",
        #"y57G11C.3"
    ]),
    'data':[
        data35[[7,6,5],0,np.newaxis,:], #GFP
        hstack((data35[[3,2,1],2:4,:],data35[[7,6,5],2,np.newaxis,:])), #gspd-1
        data35[[3,2,1],4,np.newaxis,:], #fah-1
        hstack((data35[[3,2,1],7,np.newaxis,:],data35[[7,6,5],7,np.newaxis,:])), #pmt-1
        data35[[3,2,1],9,np.newaxis,:], #tkt-1
        #hstack((data35[[3,2,1],11,np.newaxis,:],data35[[7,6,5],11,np.newaxis,:])), #Y57G11C.3
    ],
    'drug':"M9",
    'bio_rep':35
})

rep36=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "rhy-1",
    ]),
    'data':[
    data36[(7,4,2),0:4,:],#N2
    data36[(7,4,2),5:9,:],#rhy-1
    ],
    'drug':"M9",
    'bio_rep':36
})

rep37=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "rhy-1",
    ]),
    'data':[
    data37[(7,4,2),0:4,:],#N2
    data37[(7,4,2),9:,:],#rhy-1
    ],
    'drug':"M9",
    'bio_rep':37
})

rep38=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "rhy-1",
    ]),
    'data':[
    data38[(7,4,2),3:6,:],#N2
    data38[(7,4,2),8:10,:],#rhy-1
    ],
    'drug':"M9",
    'bio_rep':38
})

rep39=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "sgk-1",
        "rhy-1"
        
    ]),
    'data':[
    data39[(7,4,2),0:2,:],#N2
    data39[(7,4,2),4:6,:],#sgk-1
    data39[(7,4,2),10:12,:],#rhy-1
    ],
    'drug':"M9",
    'bio_rep':39
})

rep40=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "hif-1"
        
    ]),
    'data':[
    data40[(7,4,2),0:4,:],#N2
    data40[(7,4,2),3:6,:],#hif-1
    ],
    'drug':"M9",
    'bio_rep':40
})

rep41=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "coq-2$\Delta$6a",
        "coq-2$\Delta$6e",
        #"coq-2$\Delta$6e(AF)"
        "rict-1(mg360)",
        "rict-1(ft7)"
        
    ]),
    'data':[
    data41[(7,4,2),0:2,:],#N2
    data41[(7,4,2),2:4,:],#GUC01(delta6a)
    data41[(7,4,2),4:6,:],#GUC02(delta62)
    #data41[(7,4,2),6:8,:],#AF$\Delta$6e (AF)"
    data41[(7,4,2),8:10,:],#rict-1(mg360)
    data41[(7,4,2),10:12,:],#rict-1(ft7)
    ],
    'drug':"M9",
    'bio_rep':41
})

rep42=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "rict-1(mg360)",
        "rict-1(ft7)",
        "ire-1(ok799)"
        
    ]),
    'data':[
    data42[(7,4,2),0:2,::2],#N2
    data42[(7,4,2),6:8,::2],#rict-1(mg360)
    data42[(7,4,2),8:10,::2],#rict-1(ft7)
    data42[(7,4,2),10:12,::2],#ire-1(ok799)
    ],
    'drug':"M9",
    'bio_rep':42
})

rep43=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "rict-1(mg360)",
        "rict-1(ft7)",
        "xbp-1",
        "ire-1(zc14)"
        
    ]),
    'data':[
    hstack((data43[(7,4,2),0:2,:],data43[(7,4,2),10:12,:])),#N2
    data43[(7,4,2),2:4,:],#rict-1(mg360)
    data43[(7,4,2),4:6,:],#rict-1(ft7)
    data43[(7,4,2),6:8,:],#xbp-1
    data43[(7,4,2),8:10,:],#ire-1(zc14)
    ],
    'drug':"M9",
    'bio_rep':43
})

rep44=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "coq-2$\Delta$6a", 
        "coq-2$\Delta$6e", 
        "pept-1", 
        "ire-1(ok799)",
        "sgk-1",
        
    ]),
    'data':[
    data44[(7,4,2),0:2,:],#N2
    data44[(7,4,2),2:4,:],#GUC01(delta6a)
    data44[(7,4,2),4:6,:],#GUC02(delta6e)
    data44[(7,4,2),6:8,:],#pept-1(lg601)
    data44[(7,4,2),8:10,:],#ire-1(ok799)
    data44[(7,4,2),10:12,:],#sgk-1
    ],
    'drug':"M9",
    'bio_rep':44
})

rep45=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "ire-1(zc14)",
        
    ]),
    'data':[
    data45[(7,4,2),0:2,:],#N2
    data45[(7,4,2),9:12,:],#ire-1(zc14)
    ],
    'drug':"M9",
    'bio_rep':45
})

rep46=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "xbp-1",
        "ire-1(zc14)",
        
    ]),
    'data':[
    data46[(7,4,2),0:2,:],#N2
    data46[(7,4,2),4:6,:],#xbp-1
    data46[(7,4,2),6:8,:],#ire-1(zc14)
    ],
    'drug':"M9",
    'bio_rep':46
})

rep47=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "xbp-1"
        
    ]),
    'data':[
    data47[(7,4,2),0:2,:],#N2
    data47[(7,4,2),10:12,:],#xbp-1
    ],
    'drug':"M9",
    'bio_rep':47
})

rep48=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "sgk-1"
        
    ]),
    'data':[
    data48[(7,4,2),10:12,:],#N2
    data48[(7,4,2),0:2,:],#sgk-1
    ],
    'drug':"M9",
    'bio_rep':48
})

rep49=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "sgk-1"
        
    ]),
    'data':[
    data49[(7,4,2),0:4,:],#N2
    data49[(7,4,2),6:9,:],#sgk-1
    ],
    'drug':"M9",
    'bio_rep':49
})

rep50=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "gpd-2",
        "gpdh-1",
        "men-1",
        "pgph-2",
        "rict-1"
        
    ]),
    'data':[
    data50[(7,4,2),0:2,:],#GFP
    data50[(7,4,2),2:4,:],#gpd-2
    data50[(7,4,2),4:6,:],#gpdh-1
    data50[(7,4,2),6:8,:],#men-1
    data50[(7,4,2),8:10,:],#pgph-2
    data50[(7,4,2),10:,:],#rict-1
    ],
    'drug':"M9",
    'bio_rep':50
})

rep51=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "dhod-1",
        
    ]),
    'data':[
    hstack((data51[(7,4,2),2:4,:],data51[(7,4,2),10,np.newaxis,:])),#GFP
    hstack((data51[(7,4,2),4:6,:],data51[(7,4,2),11,np.newaxis,:])),#dhod-1
    ],
    'drug':"M9",
    'bio_rep':51
})

rep52=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "daf-15",
        "dhod-1",
        "f36a2.3",
        "prdh-1",
        "rheb-1"
        
    ]),
    'data':[
    data52[(7,4,2),0:2,:],#GFP
    data52[(7,4,2),2:4,:],#daf-15
    data52[(7,4,2),4:6,:],#dhod-1
    data52[(7,4,2),6:8,:],#f36A2.3
    data52[(7,4,2),8:10,:],#prdh-1
    data52[(7,4,2),10:,:],#rheb-1
    ],
    'drug':"M9",
    'bio_rep':52
})

rep53=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "hif-1",
        "egl-9",
        "icl-1",
        "mce-1"
        
    ]),
    'data':[
    hstack((data53[(7,4,2),0:2,:],data53[(7,4,2),10:12,:])),#N2
    data53[(7,4,2),2:4,:],#hif-1
    data53[(7,4,2),4:6,:],#egl-9
    data53[(7,4,2),6:8,:],#icl-1
    data53[(7,4,2),8:10,:],#mce-1
    ],
    'drug':"M9",
    'bio_rep':53
})

rep54=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "hif-1",
        "icl-1",
        "GFP",
        "f36a2.3",
        "mdh-1"
        
    ]),
    'data':[
    data54[(7,4,2),0:2,:],#WT
    data54[(7,4,2),2:4,:],#hif-1
    data54[(7,4,2),4:6,:],#icl-1
    data54[(7,4,2),6:8,:],#GFP
    data54[(7,4,2),8:10,:],#f36A2.3
    data54[(7,4,2),10:,:],#mdh-1
    ],
    'drug':"M9",
    'bio_rep':54
})

rep55=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "mce-1",
        "hif-1",
        "icl-1",
        "tdo-2(zg217)"
        
    ]),
    'data':[
    data55[(7,4,2),0:2,:],#WT
    data55[(7,4,2),2:4,:],#mce-1
    data55[(7,4,2),4:6,:],#hif-1
    data55[(7,4,2),6:8,:],#icl-1
    data55[(7,4,2),8:10,:],#tdo-2(zg217)
    ],
    'drug':"M9",
    'bio_rep':55
})

rep56=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "rhy-1",
        "kynu-1",
        "kynu-1;rhy-1",
        "mce-1",
        "VC40263"
        
    ]),
    'data':[
    data56[(7,4,2),0:2,:],#WT
    data56[(7,4,2),2:4,:],#rhy-1
    data56[(7,4,2),4:6,:],#kynu-1
    data56[(7,4,2),6:8,:],#kynu-1;rhy-1
    data56[(7,4,2),8:10,:],#mce-1
    data56[(7,4,2),10:,:],#VC40263
    ],
    'drug':"M9",
    'bio_rep':56
})

rep57=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "kynu-1",
        "rhy-1",
        "kynu-1;rhy-1",
        "mce-1",
        "VC40054"
        
    ]),
    'data':[
    data57[(7,4,2),0:2,:],#WT
    data57[(7,4,2),6:8,:],#kynu-1
    data57[(7,4,2),2:4,:],#rhy-1
    data57[(7,4,2),4:6,:],#kynu-1;rhy-1
    data57[(7,4,2),8:10,:],#mce-1
    data57[(7,4,2),10:,:],#VC40054
    ],
    'drug':"M9",
    'bio_rep':57
})

rep58=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "hif-1",
        "tdo-2(zg216)",
        "tdo-2(zg218)",
        
    ]),
    'data':[
    data58[(7,4,2),0:2,:],#WT
    data58[(7,4,2),2:4,:],#hif-1
    data58[(7,4,2),4:6,:],#tdo-2(zg216)
    data58[(7,4,2),6:8,:],#tdo-2(zg218)
    ],
    'drug':"M9",
    'bio_rep':58
})

rep59=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "tdo-2(zg216)",
        "tdo-2(zg217)",
        "tdo-2(zg218)",
        
    ]),
    'data':[
    data59[(7,4,2),0:3,:],#WT
    data59[(7,4,2),3:6,:],#tdo-2(zg216)
    data59[(7,4,2),6:9,:],#tdo-2(zg217)
    data59[(7,4,2),9:,:],#tdo-2(zg218)
    ],
    'drug':"M9",
    'bio_rep':59
})

rep60=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "tdo-2(zg216)",
        "tdo-2(zg217)",
        "tdo-2(zg218)",
        
    ]),
    'data':[
    hstack((data60[(7,4,2),0:3,:],data60[(7,4,2),11,np.newaxis,:])),#WT
    data60[(7,4,2),3:5,:],#tdo-2(zg216)
    data60[(7,4,2),5:8,:],#tdo-2(zg217)
    data60[(7,4,2),8:11,:],#tdo-2(zg218)
    ],
    'drug':"M9",
    'bio_rep':60
})
        
rep61=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "dhod-1",
        "f36a2.3",
        "gpdh-1",
        "prdh-1",
        "f54d5.12"
        
    ]),
    'data':[
    data61[(7,4,2),0:2,:],#GFP
    data61[(7,4,2),2:4,:],#dhod-1
    data61[(7,4,2),4:6,:],#F36A2.3
    data61[(7,4,2),6:8,:],#gpdh-1
    data61[(7,4,2),8:10,:],#prdh-1
    data61[(7,4,2),10:,:],#F56D5.12
    ],
    'drug':"M9",
    'bio_rep':61
})

rep62=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "f23c8.5",
        "mdh-1",
        "mrp-5",
        "pyc-1",
        
    ]),
    'data':[
    data62[(7,4,2),0:2,:],#GFP
    data62[(7,4,2),2:4,:],#F36A2.3
    data62[(7,4,2),4:6,:],#mdh-1
    data62[(7,4,2),6:8,:],#mrp-5
    data62[(7,4,2),8:10,:],#pyc-1
    ],
    'drug':"M9",
    'bio_rep':62
})

rep64=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "pyc-1",
        "f36a2.3",
        "prdh-1",
        "gpdh-1",
        "men-1",
        "dhod-1"
        
    ]),
    'data':[
    data64[(7,4,2),0:2,:],#GFP
    data64[(7,4,2),2:4,:],#pyc-1
    data64[(7,4,2),4:6,:],#F36A2.3
    data64[(7,4,2),6:8,:],#prdh-1
    data64[(7,4,2),8:10,:],#gpdh-1
    data64[(7,4,2),10,np.newaxis,:],#men-1
    data64[(7,4,2),11,np.newaxis,:],#dhod-2
    ],
    'drug':"M9",
    'bio_rep':64
})

rep67=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "gpdh-1",
        "mdh-1",
        "prdh-1",
        "pyc-1",
        
    ]),
    'data':[
    data67[(7,4,2),0:2,:],#GFP
    data67[(7,4,2),2:4,:],#gpdh-1
    data67[(7,4,2),4:6,:],#mdh-1
    hstack((data67[(7,4,2),6:8,:],data67[(7,4,2),10:,:])),#prdh-1
    data67[(7,4,2),8:10,:],#pyc-1
    ],
    'drug':"M9",
    'bio_rep':67
})

rep68=pd.DataFrame({
    'phenotype':pd.Categorical([
        "GFP",
        "dhod-1",
        "gpdh-1",
        "men-1",
        "pdhb-1",
        
    ]),
    'data':[
    hstack((data68[(7,4,2),0:2,:],data68[(7,4,2),10:,:])),#GFP
    data68[(7,4,2),2:4,:],#dhod-1
    data68[(7,4,2),4:6,:],#gpdh-1
    data68[(7,4,2),6:8,:],#men-1
    data68[(7,4,2),8:10,:],#pdhb-1
    ],
    'drug':"M9",
    'bio_rep':68
})

rep69=pd.DataFrame({
    'phenotype':pd.Categorical([
        "pdhb-1",
        "men-1",
        "mdh-1",
        "gpdh-1",
        "GFP",
        "f36a2.3",
        
    ]),
    'data':[
    data69[(7,4,2),0:2,:],#pdhb-1
    data69[(7,4,2),2:4,:],#men-1
    data69[(7,4,2),4:6,:],#mdh-1
    data69[(7,4,2),6:8,:],#gpdh-1
    data69[(7,4,2),8:10,:],#GFP
    data69[(7,4,2),10:,:],#F36A2.3   
    ],
    'drug':"M9",
    'bio_rep':69
})

rep70=pd.DataFrame({
    'phenotype':pd.Categorical([
        "gpdh-1",
        "pyc-1",
        "f36a2.3",
        "GFP",
        "prdh-1",
        "men-1"
        
    ]),
    'data':[
    data70[(7,4,2),0:2,:],#gpdh-1
    data70[(7,4,2),2:4,:],#pyc-1
    data70[(7,4,2),4:6,:],#F36A2.3
    data70[(7,4,2),6:8,:],#GFP
    data70[(7,4,2),8:10,:],#prdh-1
    data70[(7,4,2),10:,:],#men-1   
    ],
    'drug':"M9",
    'bio_rep':70
})

rep71=pd.DataFrame({
    'phenotype':pd.Categorical([
        "prdh-1",
        "pyc-1",
        "GFP",
        "f36a2.3",
        "men-1",
        "mdh-1"
        
    ]),
    'data':[
    data71[(7,4,2),0:2,:],#prdh-1
    data71[(7,4,2),2:4,:],#pyc-1
    data71[(7,4,2),4:6,:],#GFP
    data71[(7,4,2),6:8,:],#F36A2.3
    data71[(7,4,2),8:10,:],#men-1
    data71[(7,4,2),10:,:],#mdh-1  
    ],
    'drug':"M9",
    'bio_rep':71
})

rep72=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT", 
        "coq-2$\Delta$6a", 
        "coq-2$\Delta$6e", 
        "hyl-2", 
        "drp-1", 
        "daf-16"
        
    ]),
    'data':[
    data72[(7,4,2),0:2,:],#N2
    data72[(7,4,2),2:4,:],#GUC01(delta6a)
    data72[(7,4,2),4:6,:],#GUC02(delta6e)
    data72[(7,4,2),6:8,:],#hyl-2(gnv1)
    data72[(7,4,2),8:10,:],#drp-1(tm1108) 
    data72[(7,4,2),10:,:],#daf-16(mgDf50) 
    ],
    'drug':"M9",
    'bio_rep':72
})

rep73=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT", 
        "coq-2$\Delta$6a", 
        "coq-2$\Delta$6a",  
        "kynu-1", 
        "kynu-1;rhy-1"
        
    ]),
    'data':[
    hstack((data73[(7,4,2),0:2,:],data73[(7,4,2),6:8,:])),#N2
    data73[(7,4,2),2:4,:],#GUC01(delta6a)
    data73[(7,4,2),4:6,:],#GUC02(delta6e)
    data73[(7,4,2),8:10,:],#kynu-1
    data73[(7,4,2),10:,:],#kynu-1;rhy-1
    ],
    'drug':"M9",
    'bio_rep':73
})

rep75=pd.DataFrame({
    'phenotype':pd.Categorical([
        "WT",
        "tdo-2(zg216)",
        "tdo-2(zg217)",
        "tdo-2(zg218)",
        
    ]),
    'data':[
    data75[(7,4,2),0:3,:],#N2
    data75[(7,4,2),3:6,:],#tdo-2(zg216)
    data75[(7,4,2),6:9,:],#tdo-2(zg217)
    data75[(7,4,2),9:,:],#tdo-2(zg218)
    ],
    'drug':"M9",
    'bio_rep':75
})

rep76=pd.DataFrame({
    'phenotype':pd.Categorical([
        "prdh-1", 
        "pyc-1", 
        "GFP", 
        "f36a2.3", 
        "men-1", 
        "pdhb-1", 
        "mdh-1"
        
    ]),
    'data':[
    data76[(7,4,2),0:2,:],#prdh-1
    data76[(7,4,2),2:4,:],#pyc-1
    data76[(7,4,2),4:6,:],#GFP
    data76[(7,4,2),6:8,:],#F36A2.3
    data76[(7,4,2),8:10,:],#men-1
    data76[(7,4,2),10,np.newaxis,:],#pdhb-1
    data76[(7,4,2),11,np.newaxis,:],#mdh-1
    ],
    'drug':"M9",
    'bio_rep':76
})



In [19]:


reps=[#rep1,rep2,
    rep3,
    rep4,#rep5,
    #rep6,
      #rep7,
    #rep8,rep9,
      rep10,
      rep11,
    rep12,
    #rep13,
      #rep14,
      rep15,rep17,rep18,rep19,rep20,
      rep21,rep22,rep23,rep24,rep25,#rep26,
      rep27,rep28,rep29,#rep30,
      #rep31,
    #rep32,
    rep33,rep34,#rep35,
      rep36,rep37,rep38,rep39,rep40,rep41,rep42,rep43,rep44,
      rep45,rep46,rep47,rep48,rep49,
      rep50,rep51,rep52,#rep53,
    rep54,rep55,
      rep56,rep57,rep58,rep59,
      rep60,rep61,rep62,#rep63,
      rep64,#rep65,
      #rep66,
      #rep67,
      rep68,rep69,
      #rep70,
      rep71,rep72,rep73,
      rep75,rep76
     ]

med=True # For multiple biological reps, this determines if you want to take the median of your reps.




In [20]:
l=[]
for r in reps:
    l.append(make_longdata(r))
all_reps=pd.concat(l)

r=(all_norm(all_reps)
      .assign(log2FMS=lambda x:log2(x['mobility_score_x'])-log2(x['mobility_score_y']),
             FMS=lambda x:x['mobility_score_x']/x['mobility_score_y']))
if (med):
    r=(r
      .groupby(['concentration','phenotype','drug','bio_rep','timepoint','norm_method'])
      .median())
else:
    r=r.set_index(['concentration','phenotype','drug','bio_rep','timepoint','norm_method'])



In [21]:
r.to_csv(save_dir+today.strftime("%d%m%Y")+"_15hrdata.csv")